## Lab 03: TensorFlow vs. PyTorch

• Build and train the same small neural network model using both TensorFlow and PyTorch.

• Measure and compare training time and performance.

• Convert the trained models into lightweight formats suitable for embedded deployment: Tensor-
Flow Lite and ONNX.

## Tensorflow Implementation

In [ ]:
import tensorflow as tf
# Use the MNIST dataset of handwritten digits (28x28 grayscale images, 10 classes).
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import time

In [ ]:

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255
x_test = x_test / 255
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [ ]:
# Architecture
# • Flatten input (784 features)
# • Dense layer with 64 ReLU units
# • Output layer with 10 units (softmax for TensorFlow)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(28, 28)),              # Input shape: 28x28 pixels
    tf.keras.layers.Flatten(),                          # Flatten to 784 features
    tf.keras.layers.Dense(64, activation='relu'),       # Hidden layer with 64 neurons
    tf.keras.layers.Dense(10, activation='softmax')     # Output layer: 10 classes (digits 0–9)
])


In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',       # loss function
              metrics=['accuracy'])

start = time.time()
model.fit(x_train, y_train, epochs=5)
end = time.time()
print(f"TF Training time: {end-start:.2f} seconds")       # Output training time


Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.9832 - loss: 0.0580
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9855 - loss: 0.0487
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9868 - loss: 0.0428
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9902 - loss: 0.0343
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9911 - loss: 0.0296
TF Training time: 24.67 seconds


 Tensorflow - Test accuracy and Inference time using TensorFlow’s model.evaluate()

In [ ]:
# Measure inference time and get evaluation metrics
start_inference = time.time()

# model.evaluate runs inference on the test set and returns [loss, accuracy]
test_loss, test_accuracy = model.evaluate(x_test, y_test)

end_inference = time.time()
inference_time = end_inference - start_inference

# Print results
print(f"Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")
print(f"Test Accuracy in Percentage: {test_accuracy * 100:.2f}%")
print(f"Inference Time on Test Set: {inference_time:.2f} seconds")

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9721 - loss: 0.1013
Loss: 0.0883440151810646
Test Accuracy: 0.9753000140190125
Test Accuracy in Percentage: 97.53%
Inference Time on Test Set: 3.23 seconds


Convert the trained model to TensorFlow Lite using TFLiteConverter.

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('model.tflite', 'wb') as f:
    f.write(tflite_model)


Saved artifact at '/tmp/tmpj14hcc2t'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28), dtype=tf.float32, name='keras_tensor_8')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  134537999432016: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134537999424144: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134537999428944: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134537999423760: TensorSpec(shape=(), dtype=tf.resource, name=None)


## Pytorch Implementation

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([transforms.ToTensor(), transforms.Lambda(lambda x: x.view(-1))])
train_loader = DataLoader(datasets.MNIST(root='./data', train=True, transform=transform, download=True), batch_size=32)
test_loader = DataLoader(datasets.MNIST(root='./data', train=False, transform=transform, download=True), batch_size=1000)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 64)    #  input and output size
        self.fc2 = nn.Linear(64, 10)    #  input and output size
    def forward(self, x):
        x = F.relu(self.fc1(x))
        return self.fc2(x)

model = Net()
optimizer = optim.Adam(model.parameters())
loss_fn = nn.CrossEntropyLoss()

start = time.time()

for epoch in range(5):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for x, y in train_loader:
        optimizer.zero_grad()
        pred = model(x)
        loss = loss_fn(pred, y)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(pred.data, 1)
        total += y.size(0)
        correct += (predicted == y).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100 * correct / total
    print(f"Epoch {epoch+1}:  Accuracy = {epoch_acc} - Loss = {epoch_loss}")

end = time.time()
print(f"PyTorch Training time: {end - start:.2f} seconds")

Epoch 1:  Accuracy = 90.64666666666666 - Loss = 0.34568143301258486
Epoch 2:  Accuracy = 94.98333333333333 - Loss = 0.17169813975815973
Epoch 3:  Accuracy = 96.34833333333333 - Loss = 0.12494718434549867
Epoch 4:  Accuracy = 97.17166666666667 - Loss = 0.09756645267345011
Epoch 5:  Accuracy = 97.69166666666666 - Loss = 0.07907850412273158
PyTorch Training time: 49.84 seconds


Inference and Evaluation using PyTorch’s model.eval() + torch.no_grad().

In [ ]:
#Inference and Evaluation using PyTorch’s model.eval() + torch.no_grad().
model.eval()
correct = 0

start_time = time.time()

with torch.no_grad():
    for x, y in test_loader:
        output = model(x)
        pred = output.argmax(1)
        correct += (pred == y).sum().item()

end_time = time.time()

inference_time = end_time - start_time

print(f"Test accuracy: {correct / len(test_loader.dataset):.4f}")
print(f"Test accuracy: {(correct / len(test_loader.dataset))*100:.2f}%")
print(f"Inference Time: {inference_time:.4f} seconds")

Test accuracy: 0.9671
Test accuracy: 96.71%
Inference Time: 0.9791 seconds


convert the trained model into lightweight formats - using ONNX
1. Export the model to ONNX format.
2. Use dummy input with correct shape (e.g. torch.randn(1, 784)).
3. Save as model.onnx.

In [ ]:
# Install ONNX
!pip install onnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 116.1 MB/s eta 0:00:00


In [ ]:
dummy_input = torch.randn(1, 784)
torch.onnx.export(model, dummy_input, "model.onnx",
                  input_names=["input"], output_names=["output"])
print('Successfully saved the model as model.onnx')

Successfully saved the model as model.onnx


# TensorFlow custom training loop using tf.GradientTape

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import time

# Load and preprocess data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255
x_test = x_test / 255
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Prepare datasets
batch_size = 32         # batch size
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)

# Define model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(28, 28)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

# Define loss, optimizer, and metrics
loss_fn = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()
train_acc_metric = tf.keras.metrics.CategoricalAccuracy()
test_acc_metric = tf.keras.metrics.CategoricalAccuracy()

# Training loop
epochs = 5
start = time.time()
for epoch in range(epochs):
    print(f"\nEpoch {epoch + 1}/{epochs}")
    for step, (x_batch, y_batch) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            logits = model(x_batch, training=True)
            loss = loss_fn(y_batch, logits)
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        train_acc_metric.update_state(y_batch, logits)

        if step % 100 == 0:
            print(f"Step {step}, Loss: {loss.numpy():.4f}, Accuracy: {train_acc_metric.result().numpy():.4f}")

    print(f"Training Accuracy for epoch {epoch+1}: {train_acc_metric.result().numpy():.4f}")
    train_acc_metric.reset_state()
end = time.time()
print(f"\nTF Training time: {end - start:.2f} seconds")

# Evaluation loop
for x_batch, y_batch in test_dataset:
    test_logits = model(x_batch, training=False)
    test_acc_metric.update_state(y_batch, test_logits)

print(f"Test Accuracy: {test_acc_metric.result().numpy():.4f}")



Epoch 1/5
Step 0, Loss: 2.2509, Accuracy: 0.2188
Step 100, Loss: 0.6063, Accuracy: 0.7469
Step 200, Loss: 0.5280, Accuracy: 0.8094
Step 300, Loss: 0.8186, Accuracy: 0.8371
Step 400, Loss: 0.3434, Accuracy: 0.8569
Step 500, Loss: 0.3929, Accuracy: 0.8692
Step 600, Loss: 0.4328, Accuracy: 0.8773
Step 700, Loss: 0.2887, Accuracy: 0.8830
Step 800, Loss: 0.3304, Accuracy: 0.8890
Step 900, Loss: 0.2511, Accuracy: 0.8935
Step 1000, Loss: 0.8724, Accuracy: 0.8968
Step 1100, Loss: 0.3645, Accuracy: 0.9001
Step 1200, Loss: 0.3053, Accuracy: 0.9028
Step 1300, Loss: 0.2077, Accuracy: 0.9053
Step 1400, Loss: 0.9739, Accuracy: 0.9074
Step 1500, Loss: 0.2332, Accuracy: 0.9098
Step 1600, Loss: 0.2847, Accuracy: 0.9123
Step 1700, Loss: 0.1629, Accuracy: 0.9145
Step 1800, Loss: 0.1797, Accuracy: 0.9163
Training Accuracy for epoch 1: 0.9176

Epoch 2/5
Step 0, Loss: 0.2945, Accuracy: 0.8438
Step 100, Loss: 0.3408, Accuracy: 0.9496
Step 200, Loss: 0.2146, Accuracy: 0.9499
Step 300, Loss: 0.2037, Accuracy:

## Performance Optimization with Graph Execution using @tf.function

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import time

# Load and preprocess data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255
x_test = x_test / 255
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

# Prepare datasets
batch_size = 32
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)

# Define model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(28, 28)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

# Define loss, optimizer, and metrics
loss_fn = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()
train_acc_metric = tf.keras.metrics.CategoricalAccuracy()
test_acc_metric = tf.keras.metrics.CategoricalAccuracy()

@tf.function  # compile the function into a graph
def train_step(x_batch, y_batch):
    with tf.GradientTape() as tape:
        logits = model(x_batch, training=True)
        loss = loss_fn(y_batch, logits)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    train_acc_metric.update_state(y_batch, logits)
    return loss

# Training loop
epochs = 5
start = time.time()
for epoch in range(epochs):
    print(f"\nEpoch {epoch + 1}/{epochs}")
    for step, (x_batch, y_batch) in enumerate(train_dataset):
        loss = train_step(x_batch, y_batch)

        if step % 100 == 0:
            print(f"Step {step}, Loss: {loss.numpy():.4f}, Accuracy: {train_acc_metric.result().numpy():.4f}")

    print(f"Training Accuracy for epoch {epoch+1}: {train_acc_metric.result().numpy():.4f}")
    train_acc_metric.reset_state()
end = time.time()
print(f"\nTF Training time: {end - start:.2f} seconds")

# Evaluation loop
for x_batch, y_batch in test_dataset:
    test_logits = model(x_batch, training=False)
    test_acc_metric.update_state(y_batch, test_logits)

print(f"Test Accuracy: {test_acc_metric.result().numpy():.4f}")



Epoch 1/5
Step 0, Loss: 2.3858, Accuracy: 0.0938
Step 100, Loss: 0.7488, Accuracy: 0.7268
Step 200, Loss: 0.3294, Accuracy: 0.8002
Step 300, Loss: 0.5456, Accuracy: 0.8324
Step 400, Loss: 0.1995, Accuracy: 0.8516
Step 500, Loss: 0.4999, Accuracy: 0.8628
Step 600, Loss: 0.2850, Accuracy: 0.8717
Step 700, Loss: 0.2589, Accuracy: 0.8783
Step 800, Loss: 0.2736, Accuracy: 0.8843
Step 900, Loss: 0.2279, Accuracy: 0.8903
Step 1000, Loss: 0.1297, Accuracy: 0.8945
Step 1100, Loss: 0.4389, Accuracy: 0.8973
Step 1200, Loss: 0.1601, Accuracy: 0.9000
Step 1300, Loss: 0.2095, Accuracy: 0.9022
Step 1400, Loss: 0.1394, Accuracy: 0.9049
Step 1500, Loss: 0.2906, Accuracy: 0.9069
Step 1600, Loss: 0.1251, Accuracy: 0.9093
Step 1700, Loss: 0.2187, Accuracy: 0.9115
Step 1800, Loss: 0.0425, Accuracy: 0.9139
Training Accuracy for epoch 1: 0.9155

Epoch 2/5
Step 0, Loss: 0.0520, Accuracy: 0.9688
Step 100, Loss: 0.1222, Accuracy: 0.9474
Step 200, Loss: 0.0285, Accuracy: 0.9490
Step 300, Loss: 0.3173, Accuracy: